<a href="https://colab.research.google.com/github/Favourj-bit/gsoc_2023_pytorch_pathway_commons/blob/main/PyG_sample_data_with_InMemoryDataset_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Importing Data and Libraries

In [3]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split

In [4]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print('Using device:', device)

# if device.type != 'cuda':
#     raise SystemError('GPU device not found')

In [5]:
cBio = pd.read_csv('/content/drive/MyDrive/gsoc data/cBio.csv')

In [6]:
cBio = cBio.set_index('Sample Identifier')
cBio.index.name = None
cBio.head(2)

,Overall Survival (Months),NUDT18,SHC3,NCRNA00112,FAM49B,FMO6P,BAT2L2,SIRT1,MSLNL,ATG4A,...,SCARNA6,SNORD77,TLR10,GKN2,PCDHGB4,ZMAT5,AXIN2,LONRF1,KLHL10,C12orf36
TCGA-OR-A5J1-01,44.547457,85.0144,97.5024,0.0,671.950,0.0,2428.43,462.056,0.0,385.687,...,0.0000,0.0,12.0077,0.0,17.2911,913.535,547.55,363.112,1.9212,0.0
TCGA-OR-A5J2-01,55.133642,165.8840,92.6050,0.0,812.911,0.0,5097.30,669.977,0.0,547.980,...,0.4026,0.0,0.8053,0.0,268.9890,303.978,1793.32,592.672,0.0000,0.0


In [7]:
y = np.array(cBio['Overall Survival (Months)'], dtype=float)
X = cBio.drop('Overall Survival (Months)', axis=1)

In [8]:
path = "/content/drive/MyDrive/gsoc data/PathwayCommons12.reactome.hgnc.sif.gz"


#specify compression type because the file is compressed
df = pd.read_csv(path, sep="\t", compression="gzip", header=None, names=["Source", "InteractionType", "Target"])

In [9]:
df = df.drop(columns='InteractionType')

In [10]:
df.head()

,Source,Target
0,A1CF,APOBEC1
1,A1CF,APOBEC2
2,A1CF,APOBEC3A
3,A1CF,APOBEC3B
4,A1CF,APOBEC3C


In [11]:
all_nodes = sorted(list(set(df['Source']) | set(df['Target'])))
len(all_nodes)

12324

In [12]:
len(set(all_nodes) & set(X.columns))

9614

In [13]:
used_nodes = sorted(list(set(X.columns) & set(all_nodes)))
len(used_nodes)

9614

In [14]:
X = X.loc[:, used_nodes]
X.head(2)

,A1CF,A2M,A4GNT,AAAS,AACS,AADAC,AADAT,AAK1,AANAT,AARS,...,ZP2,ZP3,ZP4,ZRANB1,ZRSR2,ZSCAN10,ZW10,ZWILCH,ZWINT,ZZZ3
TCGA-OR-A5J1-01,0.0,10373.70,0.0,2225.26,1266.090,0.0000,121.0370,868.876,1.4409,3520.65,...,0.0,306.9160,0.0,2180.12,256.004,0.0000,253.602,183.958,146.974,596.062
TCGA-OR-A5J2-01,0.0,9844.91,0.0,1509.46,981.613,1.2079,31.0025,1627.030,0.0000,5470.94,...,0.0,99.4255,0.0,2553.48,104.684,0.4026,270.165,264.125,438.464,801.637


In [15]:
source_values = df['Source'].values
mask = np.isin(source_values, used_nodes)
df = df.loc[mask]

source_values = df['Target'].values
mask = np.isin(source_values, used_nodes)
df = df.loc[mask]
df = df.reset_index(drop=True)

In [16]:
len(set(df['Source']) | set(df['Target']))

9288

In [17]:
columns_to_select = sorted(set(df['Source']).union(set(df['Target'])))
X = X[columns_to_select]
X.head(2)

,A1CF,A2M,A4GNT,AAAS,AACS,AADAT,AAK1,AANAT,AARS,AARS2,...,ZP2,ZP3,ZP4,ZRANB1,ZRSR2,ZSCAN10,ZW10,ZWILCH,ZWINT,ZZZ3
TCGA-OR-A5J1-01,0.0,10373.70,0.0,2225.26,1266.090,121.0370,868.876,1.4409,3520.65,299.712,...,0.0,306.9160,0.0,2180.12,256.004,0.0000,253.602,183.958,146.974,596.062
TCGA-OR-A5J2-01,0.0,9844.91,0.0,1509.46,981.613,31.0025,1627.030,0.0000,5470.94,408.666,...,0.0,99.4255,0.0,2553.48,104.684,0.4026,270.165,264.125,438.464,801.637


In [18]:
source_nodes = df['Source'].tolist()
target_nodes = df['Target'].tolist()


# Create a dictionary to map each unique node to a unique index
node_to_index = {node: index for index, node in enumerate(X.columns)}

# Map the source and target nodes to their corresponding indices
source_indices = [node_to_index[node] for node in source_nodes]
target_indices = [node_to_index[node] for node in target_nodes]

In [19]:
# Convert the source and target indices to a PyTorch tensor
edge_index = torch.tensor([source_indices, target_indices], dtype=torch.long)

In [20]:
edge_index.shape

torch.Size([2, 271771])

In [22]:
# Save the tensor to a file
file_path = 'edge_index.pt'
torch.save(edge_index, file_path)

In [23]:
X.shape

(78, 9288)

In [24]:
# Set the fixed index for splitting
split_index = int(0.7 * X.shape[0])

# Split the data based on the fixed index
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

In [25]:
X_train.shape, X_test.shape

((54, 9288), (24, 9288))

In [27]:
import pandas as pd

# Save X_train to a CSV file
X_train.to_csv('X_train.csv')

# Save X_test to a CSV file
X_test.to_csv('X_test.csv')

# Save y_train to a CSV file
np.savetxt('y_train.csv', y_train, delimiter=',')

# Save y_test to a CSV file
np.savetxt('y_test.csv', y_test, delimiter=',')

In [33]:
from torch_geometric.data import InMemoryDataset, Data, download_url
import os

In [34]:
class Acc_train(InMemoryDataset):
  # Base url to download the files
    url = 'https://drive.google.com/drive/folders/14yIaLndKi-qsR0qzYg5QfUuS_mnIayKO?usp=drive_link'

    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
      super().__init__(root, transform, pre_transform, pre_filter)
      self.data, self.slices = torch.load(self.processed_paths[0])


    @property
    def raw_file_names(self):
        # List of the raw files
        return ['X_train.csv', 'y_train.csv', 'edge_index.pt']

    @property
    def processed_file_names(self):
        return 'train_data.pt'

    def download(self):
        # Download the file specified in self.url and store
        # it in self.raw_dir
        download_url(self.url, self.raw_dir)

    def process(self):

        # Load X_train from CSV file with the index
        X_train = pd.read_csv('X_train.csv', index_col=0)

        # Load y_train from CSV file
        y_train = np.loadtxt('y_train.csv', delimiter=',')

        # Load the edge_index from the file
        file_path = 'edge_index.pt'
        edge_index = torch.load(file_path)

        X_train = X_train.values

        num_patients_train = X_train.shape[0]

        # Create patient-specific graphs for the training set
        graphs_train = []
        for i in range(num_patients_train):
            node_features = X_train[i]  # Node features for the i-th patient
            target = y_train[i]  # Target label for the i-th patient
            graph_train = (node_features, edge_index, target)
            graphs_train.append(graph_train)

        # Convert graphs_train to a list of Data objects
        data_train = [Data(x=torch.tensor(graph[0].reshape(len(graphs_train[0][0]), 1)),
                           edge_index=graph[1], y=torch.tensor(graph[2]).item()) for graph in graphs_train]


        data, slices = self.collate(data_train)
        torch.save((data, slices), self.processed_paths[0])

In [35]:
class Acc_test(InMemoryDataset):
  # Base url to download the files
    url = 'https://drive.google.com/drive/folders/1SsCo5wiY-KgCYVzHHxF0SZSUWCtm6uwb?usp=drive_link'

    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
      super().__init__(root, transform, pre_transform, pre_filter)
      self.data, self.slices = torch.load(self.processed_paths[0])


    @property
    def raw_file_names(self):
        # List of the raw files
        return ['X_test.csv', 'y_test.csv', 'edge_index.pt']

    @property
    def processed_file_names(self):
        return 'test_data.pt'

    def download(self):
        # Download the file specified in self.url and store
        # it in self.raw_dir
        download_url(self.url, self.raw_dir)

    def process(self):

        # Load X_train from CSV file with the index
        X_test = pd.read_csv('X_test.csv', index_col=0)

        # Load y_train from CSV file
        y_test = np.loadtxt('y_test.csv', delimiter=',')

        # Load the edge_index from the file
        file_path = 'edge_index.pt'
        edge_index = torch.load(file_path)

        X_test = X_test.values

        num_patients_test = X_test.shape[0]

        # Create patient-specific graphs for the training set
        graphs_test = []
        for i in range(num_patients_test):
            node_features = X_test[i]  # Node features for the i-th patient
            target = y_test[i]  # Target label for the i-th patient
            graph_test = (node_features, edge_index, target)
            graphs_test.append(graph_test)

        # Convert graphs_train to a list of Data objects
        data_test = [Data(x=torch.tensor(graph[0].reshape(len(graphs_test[0][0]), 1)),
                           edge_index=graph[1], y=torch.tensor(graph[2]).squeeze()) for graph in graphs_test]


        test_data, slices = self.collate(data_test)
        torch.save((test_data, slices), self.processed_paths[0])

In [36]:
df_tr = Acc_train(root='/content/sample_data')

Using existing file 14yIaLndKi-qsR0qzYg5QfUuS_mnIayKO


In [37]:
df_test = Acc_test(root='/content/sample_data')

Using existing file 1SsCo5wiY-KgCYVzHHxF0SZSUWCtm6uwb


In [38]:
df_test

Acc_test(24)

In [39]:
df_tr

Acc_train(54)

In [40]:
# Access the attributes of a specific data object in the training set
sample = df_test[0]  # Get the first data object
print(sample)  # Print the data object

# Access the node features, edge indices, and target label
node_features = sample.x
edge_index = sample.edge_index
target = sample.y

print(node_features)  # Print the node features
print(edge_index)  # Print the edge indices
print(target)  # Print the target label

Data(x=[9288, 1], edge_index=[2, 271771], y=[1])
tensor([[    0.0000],
        [16494.5000],
        [    0.0000],
        ...,
        [   83.4073],
        [  106.4770],
        [  403.7270]], dtype=torch.float64)
tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])
tensor([23.6381])
